In [1]:
import os
import json
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "replace with your credentials"
!pip install Flask
!pip install requests
!pip install google-cloud-bigquery
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

from collections import defaultdict
import datetime
import json

# Define the date ranges for each president's term in Unix timestamp format
obama_start_date = int(datetime.datetime(2009, 1, 20, 0, 0, 0).timestamp())
obama_end_date = int(datetime.datetime(2017, 1, 20, 0, 0, 0).timestamp())
trump_start_date = int(datetime.datetime(2017, 1, 20, 0, 0, 0).timestamp())
trump_end_date = int(datetime.datetime(2021, 1, 20, 0, 0, 0).timestamp())
biden_start_date = int(datetime.datetime(2021, 1, 20, 0, 0, 0).timestamp())
biden_end_date = int(datetime.datetime.utcnow().timestamp())

# Query to filter rows by date ranges and calculate sentiment counts
query = f"""
SELECT
    CASE
        WHEN created_utc >= {obama_start_date} AND created_utc <= {obama_end_date} THEN 'Barack Obama'
        WHEN created_utc >= {trump_start_date} AND created_utc <= {trump_end_date} THEN 'Donald Trump'
        WHEN created_utc >= {biden_start_date} THEN 'Joe Biden'
    END AS president,
    CASE
        WHEN prediction = 0.0 THEN 'negative'
        WHEN prediction = 1.0 THEN 'positive'
        WHEN prediction = 2.0 THEN 'neutral'
    END AS sentiment,
    COUNT(*) AS count
FROM
    famous-athlete-386604.reddit.comment
WHERE
    created_utc >= {obama_start_date} AND created_utc <= UNIX_SECONDS(CURRENT_TIMESTAMP())  -- Adjust the date range accordingly
GROUP BY
    president, sentiment
ORDER BY
    president, sentiment
"""

# Run the query
query_job = client.query(query)

# Get the results
results = query_job.result()

# Store the results in a dictionary
data = defaultdict(dict)
for row in results:
    president = row['president']
    sentiment = row['sentiment']
    count = row['count']
    data[president][sentiment] = count

# Convert the data dictionary to the desired format
sentiment_counts = {
    "Barack Obama": {
        "negative": data["Barack Obama"].get("negative", 0),
        "neutral": data["Barack Obama"].get("neutral", 0),
        "positive": data["Barack Obama"].get("positive", 0)
    },
    "Donald Trump": {
        "negative": data["Donald Trump"].get("negative", 0),
        "neutral": data["Donald Trump"].get("neutral", 0),
        "positive": data["Donald Trump"].get("positive", 0)
    },
    "Joe Biden": {
        "negative": data["Joe Biden"].get("negative", 0),
        "neutral": data["Joe Biden"].get("neutral", 0),
        "positive": data["Joe Biden"].get("positive", 0)
    }
}

# Clear the contents of sentimentCounts.json if it exists
if os.path.exists('sentimentCounts.json'):
    os.remove('sentimentCounts.json')

# Write the sentiment_counts dictionary to sentimentCounts.json
with open('sentimentCounts.json', 'w') as file:
    json.dump(sentiment_counts, file)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
